In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf

if __name__ == "__main__":
    spark = SparkSession.builder.appName("Chapter8").getOrCreate()
    sc = spark.sparkContext

In [ ]:
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])])\
  .toDF("id", "name", "graduate_program", "spark_status")

graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley")])\
  .toDF("id", "degree", "department", "school")
sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")])\
  .toDF("id", "status")

In [ ]:
joinExpression = person["graduate_program"] == graduateProgram['id']

#Inner

joinType = "inner"
person.join(graduateProgram, joinExpression,joinType).show()

#outer
joinType = "outer"
person.join(graduateProgram, joinExpression,joinType).show()

#Left outer
joinType = "left_outer"
graduateProgram.join(person, joinExpression,joinType).show()

#Right Outer
joinType = "right_outer"
person.join(graduateProgram, joinExpression,joinType).show()

#Join in Complex Types

from pyspark.sql.functions import expr

person.withColumnRenamed("id", "personId")\
  .join(sparkStatus, expr("array_contains(spark_status, id)")).show()

#three overcome the challenges for join
#Use string instead of boolean on="sample"
person.join(gradProgramDupe,"graduate_program").select("graduate_program").show()
#Drop the column after the join
person.join(gradProgramDupe, joinExpr).drop(person.col("graduate_program"))
  .select("graduate_program").show()
#Rename the clolumn before the join

In [ ]:
#Spark join performance

#Shuffle Join - very expensive - need to do intelligent partitioning
#Broacast join - no expensive - Big table to small table - broadcast small table to all worker node in order to avoid shuffle

from 90 pyspark.sql.functions import broadcast
#perform broadcast function use only small dataframe
person.join(broadcast(graduateProgram), joinExpr).explain()